In [ ]:
!pip install -q evaluate transformers==4.28.1
!pip install -U -q datasets
!pip install -q torchaudio==0.12.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!add-apt-repository -y ppa:savoury1/ffmpeg4
!apt-get -qq install -y ffmpeg
!pip install -q mlflow
!pip install pydub
!pip install IPython
!pip install imblearn==0.0


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1 requires torch==1.12.1, but you have torch 1.12.0+cu113 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
/bin/bash: add-apt-repository: command not found
E: Unable to locate package ffmpeg

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To upda

In [ ]:
#imports
import pandas as pd
import gc
import re
import numpy as np
import os

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import torch
import torchaudio
import datasets
import transformers
from sklearn.model_selection import train_test_split

In [ ]:
!unzip "recordings (1).zip"

  inflating: recordings/english/english579.mp3  
  inflating: recordings/english/english58.mp3  
  inflating: recordings/english/english59.mp3  
  inflating: recordings/english/english6.mp3  
  inflating: recordings/english/english60.mp3  
  inflating: recordings/english/english61.mp3  
  inflating: recordings/english/english62.mp3  
  inflating: recordings/english/english63.mp3  
  inflating: recordings/english/english64.mp3  
  inflating: recordings/english/english65.mp3  
  inflating: recordings/english/english66.mp3  
  inflating: recordings/english/english67.mp3  
  inflating: recordings/english/english68.mp3  
  inflating: recordings/english/english69.mp3  
  inflating: recordings/english/english7.mp3  
  inflating: recordings/english/english70.mp3  
  inflating: recordings/english/english71.mp3  
  inflating: recordings/english/english72.mp3  
  inflating: recordings/english/english73.mp3  
  inflating: recordings/english/english74.mp3  
  inflating: recordings/english/english75

In [ ]:
def language_files_to_list2(directory):
  # iterate over files in that directory
  recordings = []
  for filename in os.scandir(directory):
    # add to recordings list
    recordings.append(filename.path)
  return recordings

In [ ]:
!ls /datasets/google-drive

afrikaans  cantonese  hausa	 kurdish     portuguese  tagalog     yoruba
albanian   dutch      hindi	 macedonian  punjabi	 taiwanese
amharic    english    hungarian  mandarian   romanian	 thai
arabic	   farsi      italian	 mongolian   russian	 turkish
armenian   french     japanese	 nepali      serbian	 ukrainian
bengali    german     kiswahili  pashto      spanish	 urdu
bulgarian  greek      korean	 polish      swedish	 vietnamese


In [ ]:
dict_of_recordings = {'filename':[],
                      'accent':[]}

for root, dirs, files in os.walk('/datasets/google-drive/recordings/recordings'):
   for dir in dirs:
    #print(name) # just the language
    #print(os.path.join(root, name)) # language directories

    lang_list = language_files_to_list2(str(os.path.join(root, dir)))
    filename_list = [i for i in lang_list]
    accent_list = [dir for i in range(len(lang_list))]

    dict_of_recordings['filename'].extend(filename_list)
    dict_of_recordings['accent'].extend(accent_list)

print(dict_of_recordings['accent'])
print(dict_of_recordings)

['macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'macedonian', 'hungarian', 'hungarian', 'hungarian', 'hungarian', 'hungarian', 'hungarian', 'hungarian', 'hungarian', 'hungarian', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'swedish', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'serbian', 'korean', 'korean', 'korean', 'korean', 'korean', 'korean', 'korean', 'korean', 'korean', 'korean', 

In [ ]:
df = pd.DataFrame(data=dict_of_recordings)

In [ ]:
df.head()

,filename,accent
0,/datasets/google-drive/recordings/recordings/m...,macedonian
1,/datasets/google-drive/recordings/recordings/m...,macedonian
2,/datasets/google-drive/recordings/recordings/m...,macedonian
3,/datasets/google-drive/recordings/recordings/m...,macedonian
4,/datasets/google-drive/recordings/recordings/m...,macedonian


In [ ]:
#df.dtypes
df['accent'].value_counts()

english       579
spanish       162
arabic        102
mandarian      65
french         63
korean         52
portuguese     48
russian        48
dutch          47
turkish        37
german         36
polish         34
italian        33
japanese       27
macedonian     26
cantonese      23
farsi          23
vietnamese     22
swedish        20
romanian       20
amharic        20
serbian        18
bulgarian      18
tagalog        18
hindi          18
bengali        17
urdu           16
greek          15
thai           15
nepali         13
ukrainian      11
kurdish        10
punjabi        10
pashto         10
hausa           9
kiswahili       9
mongolian       9
hungarian       9
albanian        9
armenian        8
taiwanese       8
afrikaans       5
yoruba          5
Name: accent, dtype: int64

In [ ]:
from collections import Counter
labels = [lang for lang, _ in Counter(df['accent']).most_common(10)]

label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

print(id2label, '\n\n', label2id)

{0: 'english', 1: 'spanish', 2: 'arabic', 3: 'mandarian', 4: 'french', 5: 'korean', 6: 'russian', 7: 'portuguese', 8: 'dutch', 9: 'turkish'} 

 {'english': 0, 'spanish': 1, 'arabic': 2, 'mandarian': 3, 'french': 4, 'korean': 5, 'russian': 6, 'portuguese': 7, 'dutch': 8, 'turkish': 9}


In [ ]:
# creating a new dataset to preserve the original in case
df2 = df.copy()
df2 = df2.loc[df2['accent'].isin(labels)]
df2['label'] = df2['accent'].apply(lambda x: label2id[x])
df2 = df2[['filename', 'label']]

In [ ]:
df2

,filename,label
73,/datasets/google-drive/recordings/recordings/k...,5
74,/datasets/google-drive/recordings/recordings/k...,5
75,/datasets/google-drive/recordings/recordings/k...,5
76,/datasets/google-drive/recordings/recordings/k...,5
77,/datasets/google-drive/recordings/recordings/k...,5
...,...,...
1676,/datasets/google-drive/recordings/recordings/d...,8
1677,/datasets/google-drive/recordings/recordings/d...,8
1678,/datasets/google-drive/recordings/recordings/d...,8
1679,/datasets/google-drive/recordings/recordings/d...,8


In [ ]:
# random undersampling of all but minority class
# Set the seed value for experiment reproducibility.

rus = RandomUnderSampler(random_state=42, sampling_strategy='not minority')
y = df2[['label']]
x = df2.drop(['label'], axis=1)
#x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.20,stratify=y,random_state=42)
#y_train.value_counts()
df2, y_resampled = rus.fit_resample(df2, y)
del y
df2['label'] = y_resampled
del y_resampled
gc.collect()

239

In [ ]:
df2['label'].value_counts()

0    37
1    37
2    37
3    37
4    37
5    37
6    37
7    37
8    37
9    37
Name: label, dtype: int64

In [ ]:
RATE_HZ = 16000
MAX_LENGTH = 80000
def get_transform_audio(file):
    audio,rate = torchaudio.load(file) # load the file
    transform = torchaudio.transforms.Resample(rate,RATE_HZ) # resample to 16000hertz
    audio = transform(audio).squeeze(0).numpy()
    audio = audio[:MAX_LENGTH] # truncate to first part of audio to save RAM
    return audio
df2['audio'] = df2['filename'].progress_apply(get_transform_audio)
gc.collect()

  0%|          | 1/370 [00:00<01:10,  5.26it/s]


RuntimeError: Failed to load audio from /datasets/google-drive/recordings/recordings/english/english422.mp3

In [ ]:
# dropping the filename column
%%time
df2 = df2.drop(['filename'], axis=1)
gc.collect()

In [ ]:
from datasets import Dataset
# Hugging Face's Datasets library is a powerful tool for data scientists
# offering a more memory-efficient way to handle large datasets.
dd = Dataset.from_pandas(df2)
gc.collect()

In [ ]:
dd = dd.train_test_split(test_size=0.25)
gc.collect()
dd

In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

model_str = "facebook/wav2vec2-base-960h"
feature_extractor=AutoFeatureExtractor.from_pretrained(model_str)
model=AutoModelForAudioClassification.from_pretrained(model_str,num_labels=len(labels))
model.config.id2label = id2label
# number of trainable parameters
print(model.num_parameters(only_trainable=True)/1e6)

In [ ]:
def preprocess_function(batch):
    inputs = feature_extractor(batch['audio'], sampling_rate=RATE_HZ, max_length=MAX_LENGTH, truncation=True)
    inputs['input_values'] = inputs['input_values'][0]
    return inputs

dd['train'] = dd['train'].map(preprocess_function, remove_columns="audio", batched=False)
gc.collect()
dd['test'] = dd['test'].map(preprocess_function, remove_columns="audio", batched=False)
gc.collect()

In [ ]:
#import evaluate

#accuracy = evaluate.load("accuracy")

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
def compute_metrics(eval_pred):
    # Compute the ROC AUC score
    predictions = eval_pred.predictions
    predictions = np.exp(predictions)/np.exp(predictions).sum(axis=1, keepdims=True)
    label_ids = eval_pred.label_ids
    roc_auc = roc_auc_score(label_ids, predictions, average='macro', multi_class='ovr')

    # Calculate accuracy using the loaded accuracy metric
    acc_score1 = accuracy_score(y_true=label_ids, y_pred=predictions.argmax(axis=1))
    #acc_score = accuracy.compute(predictions=predictions.argmax(axis=1), references=label_ids)['accuracy']

    return {
        "roc_auc": roc_auc,
        "accuracy": acc_score1
    }

In [ ]:
from transformers import TrainingArguments, Trainer
batch_size=8
warmup_steps=50
weight_decay=0.02
num_train_epochs=10
model_name = "english_accents_classification"
training_args = TrainingArguments(
    output_dir=model_name,
    logging_dir='./logs',
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-5, # 3e-5
    logging_strategy='steps',
    logging_first_step=True,
    load_best_model_at_end=True,
    logging_steps=1,
    evaluation_strategy='epoch',
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    eval_steps=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    save_strategy='epoch',
    save_total_limit=1, # save fewer checkpoints to limit used space
    report_to="mlflow",  # log to mlflow
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dd["train"],
    eval_dataset=dd["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [ ]:
# initial model evaluation
trainer.evaluate()

In [ ]:
# model training
trainer.train()

In [ ]:
# final model evaluation
trainer.evaluate()

In [ ]:
trainer.save_model()

In [ ]:
# finally, save the model to Huggingface
from huggingface_hub import notebook_login
notebook_login()

from huggingface_hub import HfApi
api = HfApi()
repo_id = f"jadao/{model_name}"
try:
    api.create_repo(repo_id)
except:
    print(f"Repo {repo_id} already exists")
api.upload_folder(
    folder_path=model_name,
    path_in_repo = ".",
    repo_id=repo_id,
    repo_type="model"
)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fb7695b1-5fd9-42c6-b2ce-16b15b4ec4c5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>